# ECG Ensemble 1D-CNN Server Side
This code is the server part of ECG ensemble 1D-CNN model for **multi** client and a server.

## Setting variables

In [1]:
epochs = 1
users = 3 # number of users

## Import required packages

In [2]:
import os
import socket
import struct
import pickle
from threading import Thread
from threading import Lock
import time
import sys


import h5py
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam

## Set CUDA

In [3]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

torch.manual_seed(777)
if device =="cuda:0":
    torch.cuda.manual_seed_all(777)

In [4]:
root_path = '../models/'

## datasets

In [5]:
class ECG(Dataset):
    def __init__(self, train=True):
        if train:
            with h5py.File(os.path.join(root_path, 'ecg_data', 'train_ecg.hdf5'), 'r') as hdf:
                self.x = hdf['x_train'][:]
                self.y = hdf['y_train'][:]
        else:
            with h5py.File(os.path.join(root_path, 'ecg_data', 'test_ecg.hdf5'), 'r') as hdf:
                self.x = hdf['x_test'][:]
                self.y = hdf['y_test'][:]
    
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, idx):
        return torch.tensor(self.x[idx], dtype=torch.float), torch.tensor(self.y[idx])

### Set batch size

In [6]:
batch_size = 32

## Make train and test dataset batch generator

In [7]:
train_dataset = ECG(train=True)
test_dataset = ECG(train=False)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [8]:
server_models = [0] * users
original_models = [0] * users

## Define ECG server model
Server side has **1 convolutional layer** and **2 fully connected layers**.


In [9]:
class EcgServer1(nn.Module):
    def __init__(self):
        super(EcgServer1, self).__init__()
#         self.conv1 = nn.Conv1d(1, 16, 7, padding=3)  # 128 x 16
#         self.relu1 = nn.LeakyReLU()
#         self.pool1 = nn.MaxPool1d(2)  # 64 x 16
#         self.conv2 = nn.Conv1d(16, 16, 5, padding=2)  # 64 x 16
#         self.relu2 = nn.LeakyReLU()
        self.conv3 = nn.Conv1d(16, 16, 5, padding=2)  # 64 x 16
        self.relu3 = nn.LeakyReLU()
        self.conv4 = nn.Conv1d(16, 16, 5, padding=2)  # 64 x 16
        self.relu4 = nn.LeakyReLU()
        self.pool4 = nn.MaxPool1d(2)  # 32 x 16
        self.linear5 = nn.Linear(32 * 16, 128)
        self.relu5 = nn.LeakyReLU()
        self.linear6 = nn.Linear(128, 5)
        self.softmax6 = nn.Softmax(dim=1)
    
    def forward(self, x):
#         x = self.conv1(x)
#         x = self.relu1(x)
#         x = self.pool1(x)
#         x = self.conv2(x)
#         x = self.relu2(x)
        x = self.conv3(x)
        x = self.relu3(x)
        x = self.conv4(x)
        x = self.relu4(x)
        x = self.pool4(x)
        x = x.view(-1, 32 * 16)
        x = self.linear5(x)
        x = self.relu5(x)
        x = self.linear6(x)
        x = self.softmax6(x)
        return x   
    
class Ecgnet1(nn.Module):
    def __init__(self):
        super(Ecgnet1, self).__init__()
        self.conv1 = nn.Conv1d(1, 16, 7, padding=3)  # 128 x 16
        self.relu1 = nn.LeakyReLU()
        self.pool1 = nn.MaxPool1d(2)  # 64 x 16
        self.conv2 = nn.Conv1d(16, 16, 5, padding=2)  # 64 x 16
        self.relu2 = nn.LeakyReLU()
        self.conv3 = nn.Conv1d(16, 16, 5, padding=2)  # 64 x 16
        self.relu3 = nn.LeakyReLU()
        self.conv4 = nn.Conv1d(16, 16, 5, padding=2)  # 64 x 16
        self.relu4 = nn.LeakyReLU()
        self.pool4 = nn.MaxPool1d(2)  # 32 x 16
        self.linear5 = nn.Linear(32 * 16, 128)
        self.relu5 = nn.LeakyReLU()
        self.linear6 = nn.Linear(128, 5)
        self.softmax6 = nn.Softmax(dim=1)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.conv3(x)
        x = self.relu3(x)
        x = self.conv4(x)
        x = self.relu4(x)
        x = self.pool4(x)
        x = x.view(-1, 32 * 16)
        x = self.linear5(x)
        x = self.relu5(x)
        x = self.linear6(x)
        x = self.softmax6(x)
        return x 

In [10]:
server_models[0] = EcgServer1().to(device)
print(server_models[0])

original_models[0] = Ecgnet1().to(device)
print(original_models[0])

EcgServer1(
  (conv3): Conv1d(16, 16, kernel_size=(5,), stride=(1,), padding=(2,))
  (relu3): LeakyReLU(negative_slope=0.01)
  (conv4): Conv1d(16, 16, kernel_size=(5,), stride=(1,), padding=(2,))
  (relu4): LeakyReLU(negative_slope=0.01)
  (pool4): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (linear5): Linear(in_features=512, out_features=128, bias=True)
  (relu5): LeakyReLU(negative_slope=0.01)
  (linear6): Linear(in_features=128, out_features=5, bias=True)
  (softmax6): Softmax(dim=1)
)
Ecgnet1(
  (conv1): Conv1d(1, 16, kernel_size=(7,), stride=(1,), padding=(3,))
  (relu1): LeakyReLU(negative_slope=0.01)
  (pool1): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv1d(16, 16, kernel_size=(5,), stride=(1,), padding=(2,))
  (relu2): LeakyReLU(negative_slope=0.01)
  (conv3): Conv1d(16, 16, kernel_size=(5,), stride=(1,), padding=(2,))
  (relu3): LeakyReLU(negative_slope=0.01)
  (conv4): Conv1d(16, 16, kernel_size=(5

In [11]:
# from torchsummary import summary

# print('ECG 1D CNN server')
# summary(server_models[0], (16, 65))

In [12]:
class EcgServer2(nn.Module):
    def __init__(self):
        super(EcgServer2, self).__init__()
#         self.conv1 = nn.Conv1d(1, 16, 7, padding=3)  # 128 x 16
#         self.relu1 = nn.LeakyReLU()
#         self.pool1 = nn.MaxPool1d(2)  # 64 x 16
#         self.conv2 = nn.Conv1d(16, 16, 5, padding=2)  # 64 x 16
#         self.relu2 = nn.LeakyReLU()
        self.conv3 = nn.Conv1d(16, 16, 5, padding=2)  # 64 x 16
        self.relu3 = nn.LeakyReLU()
        self.conv4 = nn.Conv1d(16, 16, 5, padding=2)  # 64 x 16
        self.relu4 = nn.LeakyReLU()
        self.conv5 = nn.Conv1d(16, 16, 5, padding=2)  # 64 x 16
        self.relu5 = nn.LeakyReLU()
        self.pool5 = nn.MaxPool1d(2)  # 32 x 16
        self.linear6 = nn.Linear(32 * 16, 128)
        self.relu6 = nn.LeakyReLU()
        self.linear7 = nn.Linear(128, 5)
        self.softmax7 = nn.Softmax(dim=1)
    
    def forward(self, x):
#         x = self.conv1(x)
#         x = self.relu1(x)
#         x = self.pool1(x)
#         x = self.conv2(x)
#         x = self.relu2(x)
        x = self.conv3(x)
        x = self.relu3(x)
        x = self.conv4(x)
        x = self.relu4(x)
        x = self.conv5(x)
        x = self.relu5(x)
        x = self.pool5(x)
        x = x.view(-1, 32 * 16)
        x = self.linear6(x)
        x = self.relu6(x)
        x = self.linear7(x)
        x = self.softmax7(x)
        return x   
    
class Ecgnet2(nn.Module):
    def __init__(self):
        super(Ecgnet2, self).__init__()
        self.conv1 = nn.Conv1d(1, 16, 7, padding=3)  # 128 x 16
        self.relu1 = nn.LeakyReLU()
        self.pool1 = nn.MaxPool1d(2)  # 64 x 16
        self.conv2 = nn.Conv1d(16, 16, 5, padding=2)  # 64 x 16
        self.relu2 = nn.LeakyReLU()
        self.conv3 = nn.Conv1d(16, 16, 5, padding=2)  # 64 x 16
        self.relu3 = nn.LeakyReLU()
        self.conv4 = nn.Conv1d(16, 16, 5, padding=2)  # 64 x 16
        self.relu4 = nn.LeakyReLU()
        self.conv5 = nn.Conv1d(16, 16, 5, padding=2)  # 64 x 16
        self.relu5 = nn.LeakyReLU()
        self.pool5 = nn.MaxPool1d(2)  # 32 x 16
        self.linear6 = nn.Linear(32 * 16, 128)
        self.relu6 = nn.LeakyReLU()
        self.linear7 = nn.Linear(128, 5)
        self.softmax7 = nn.Softmax(dim=1)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.conv3(x)
        x = self.relu3(x)
        x = self.conv4(x)
        x = self.relu4(x)
        x = self.conv5(x)
        x = self.relu5(x)
        x = self.pool5(x)
        x = x.view(-1, 32 * 16)
        x = self.linear6(x)
        x = self.relu6(x)
        x = self.linear7(x)
        x = self.softmax7(x)
        return x   

In [13]:
server_models[1] = EcgServer2().to(device)
print(server_models[1])

original_models[1] = Ecgnet2().to(device)
print(original_models[1])

EcgServer2(
  (conv3): Conv1d(16, 16, kernel_size=(5,), stride=(1,), padding=(2,))
  (relu3): LeakyReLU(negative_slope=0.01)
  (conv4): Conv1d(16, 16, kernel_size=(5,), stride=(1,), padding=(2,))
  (relu4): LeakyReLU(negative_slope=0.01)
  (conv5): Conv1d(16, 16, kernel_size=(5,), stride=(1,), padding=(2,))
  (relu5): LeakyReLU(negative_slope=0.01)
  (pool5): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (linear6): Linear(in_features=512, out_features=128, bias=True)
  (relu6): LeakyReLU(negative_slope=0.01)
  (linear7): Linear(in_features=128, out_features=5, bias=True)
  (softmax7): Softmax(dim=1)
)
Ecgnet2(
  (conv1): Conv1d(1, 16, kernel_size=(7,), stride=(1,), padding=(3,))
  (relu1): LeakyReLU(negative_slope=0.01)
  (pool1): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv1d(16, 16, kernel_size=(5,), stride=(1,), padding=(2,))
  (relu2): LeakyReLU(negative_slope=0.01)
  (conv3): Conv1d(16, 16, kernel_size=(5

In [14]:
# from torchsummary import summary

# print('ECG 1D CNN server')
# summary(server_models[1], (16, 65))   # 16, 65

In [15]:
class EcgServer3(nn.Module):
    def __init__(self):
        super(EcgServer3, self).__init__()
#         self.conv1 = nn.Conv1d(1, 16, 7, padding=3)  # 128 x 16
#         self.relu1 = nn.LeakyReLU()
#         self.pool1 = nn.MaxPool1d(2)  # 64 x 16
#         self.conv2 = nn.Conv1d(16, 16, 5, padding=2)  # 64 x 16
#         self.relu2 = nn.LeakyReLU()
        self.conv3 = nn.Conv1d(16, 16, 5, padding=2)  # 64 x 16
        self.relu3 = nn.LeakyReLU()
        self.conv4 = nn.Conv1d(16, 16, 5, padding=2)  # 64 x 16
        self.relu4 = nn.LeakyReLU()
        self.conv5 = nn.Conv1d(16, 16, 5, padding=2)  # 64 x 16
        self.relu5 = nn.LeakyReLU()
        self.conv6 = nn.Conv1d(16, 16, 5, padding=2)  # 64 x 16
        self.relu6 = nn.LeakyReLU()
        self.pool6 = nn.MaxPool1d(2)  # 32 x 16
        self.linear7 = nn.Linear(32 * 16, 128)
        self.relu7 = nn.LeakyReLU()
        self.linear8 = nn.Linear(128, 5)
        self.softmax8 = nn.Softmax(dim=1)
    
    def forward(self, x):
#         x = self.conv1(x)
#         x = self.relu1(x)
#         x = self.pool1(x)
#         x = self.conv2(x)
#         x = self.relu2(x)
        x = self.conv3(x)
        x = self.relu3(x)
        x = self.conv4(x)
        x = self.relu4(x)
        x = self.conv5(x)
        x = self.relu5(x)
        x = self.conv6(x)
        x = self.relu6(x)
        x = self.pool6(x)
        x = x.view(-1, 32 * 16)
        x = self.linear7(x)
        x = self.relu7(x)
        x = self.linear8(x)
        x = self.softmax8(x)
        return x        
    
class Ecgnet3(nn.Module):
    def __init__(self):
        super(Ecgnet3, self).__init__()
        self.conv1 = nn.Conv1d(1, 16, 7, padding=3)  # 128 x 16
        self.relu1 = nn.LeakyReLU()
        self.pool1 = nn.MaxPool1d(2)  # 64 x 16
        self.conv2 = nn.Conv1d(16, 16, 5, padding=2)  # 64 x 16
        self.relu2 = nn.LeakyReLU()
        self.conv3 = nn.Conv1d(16, 16, 5, padding=2)  # 64 x 16
        self.relu3 = nn.LeakyReLU()
        self.conv4 = nn.Conv1d(16, 16, 5, padding=2)  # 64 x 16
        self.relu4 = nn.LeakyReLU()
        self.conv5 = nn.Conv1d(16, 16, 5, padding=2)  # 64 x 16
        self.relu5 = nn.LeakyReLU()
        self.conv6 = nn.Conv1d(16, 16, 5, padding=2)  # 64 x 16
        self.relu6 = nn.LeakyReLU()
        self.pool6 = nn.MaxPool1d(2)  # 32 x 16
        self.linear7 = nn.Linear(32 * 16, 128)
        self.relu7 = nn.LeakyReLU()
        self.linear8 = nn.Linear(128, 5)
        self.softmax8 = nn.Softmax(dim=1)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.conv3(x)
        x = self.relu3(x)
        x = self.conv4(x)
        x = self.relu4(x)
        x = self.conv5(x)
        x = self.relu5(x)
        x = self.conv6(x)
        x = self.relu6(x)
        x = self.pool6(x)
        x = x.view(-1, 32 * 16)
        x = self.linear7(x)
        x = self.relu7(x)
        x = self.linear8(x)
        x = self.softmax8(x)
        return x        

In [16]:
server_models[2] = EcgServer3().to(device)
print(server_models[2])

original_models[2] = Ecgnet3().to(device)
print(original_models[2])

EcgServer3(
  (conv3): Conv1d(16, 16, kernel_size=(5,), stride=(1,), padding=(2,))
  (relu3): LeakyReLU(negative_slope=0.01)
  (conv4): Conv1d(16, 16, kernel_size=(5,), stride=(1,), padding=(2,))
  (relu4): LeakyReLU(negative_slope=0.01)
  (conv5): Conv1d(16, 16, kernel_size=(5,), stride=(1,), padding=(2,))
  (relu5): LeakyReLU(negative_slope=0.01)
  (conv6): Conv1d(16, 16, kernel_size=(5,), stride=(1,), padding=(2,))
  (relu6): LeakyReLU(negative_slope=0.01)
  (pool6): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (linear7): Linear(in_features=512, out_features=128, bias=True)
  (relu7): LeakyReLU(negative_slope=0.01)
  (linear8): Linear(in_features=128, out_features=5, bias=True)
  (softmax8): Softmax(dim=1)
)
Ecgnet3(
  (conv1): Conv1d(1, 16, kernel_size=(7,), stride=(1,), padding=(3,))
  (relu1): LeakyReLU(negative_slope=0.01)
  (pool1): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv1d(16, 16, kernel_size=(5

## client

In [17]:
class Ecgclient(nn.Module):
    def __init__(self):
        super(Ecgclient, self).__init__()
        self.conv1 = nn.Conv1d(1, 16, 7, padding=3)  # 128 x 16
        self.relu1 = nn.LeakyReLU()
        self.pool1 = nn.MaxPool1d(2)  # 64 x 16
        self.conv2 = nn.Conv1d(16, 16, 5, padding=2)  # 64 x 16
        self.relu2 = nn.LeakyReLU()
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        return x   

In [18]:
ecg_client = Ecgclient().to(device)
print(ecg_client)

Ecgclient(
  (conv1): Conv1d(1, 16, kernel_size=(7,), stride=(1,), padding=(3,))
  (relu1): LeakyReLU(negative_slope=0.01)
  (pool1): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv1d(16, 16, kernel_size=(5,), stride=(1,), padding=(2,))
  (relu2): LeakyReLU(negative_slope=0.01)
)


In [19]:
# from torchsummary import summary

# print('ECG 1D CNN client')
# summary(ecg_client, (1, 130))

### Set other hyperparameters in the model
Hyperparameters here should be same with the client side.

In [20]:
import copy

criterion = nn.CrossEntropyLoss()
lr = 0.001
optimizer_server_list = []
for model in server_models:
    optimizer_server_list.append(Adam(model.parameters(), lr=lr))

clientsoclist = [0] * users
val_acc = [[] for i in range(users)]
train_acc = [[] for i in range(users)]

client_weights = []
weight_count = 0
weights = copy.deepcopy(ecg_client.state_dict())
for _ in range(users):
    client_weights.append(weights)

start_time = 0
lock = Lock()
    
###########################################################################

############################################################################

In [21]:
total_sendsize_list = []
total_receivesize_list = []

client_sendsize_list = [[] for i in range(users)]
client_receivesize_list = [[] for i in range(users)]

train_sendsize_list = [] 
train_receivesize_list = []

## Socket initialization
### Set host address and port number

### Required socket functions

In [22]:
def send_msg(sock, msg):
    # prefix each message with a 4-byte length in network byte order
    msg = pickle.dumps(msg)
    l_send = len(msg)
    msg = struct.pack('>I', l_send) + msg
    sock.sendall(msg)
    return l_send

def recv_msg(sock):
    # read message length and unpack it into an integer
    raw_msglen = recvall(sock, 4)
    if not raw_msglen:
        return None
    msglen = struct.unpack('>I', raw_msglen)[0]
    # read the message data
    msg =  recvall(sock, msglen)
    msg = pickle.loads(msg)
    return msg, msglen

def recvall(sock, n):
    # helper function to receive n bytes or return None if EOF is hit
    data = b''
    while len(data) < n:
        packet = sock.recv(n - len(data))
        if not packet:
            return None
        data += packet
    return data


In [23]:
def run_thread(func, num_user):
    global clientsoclist
    global start_time
    thrs = []
    for i in range(num_user):
        conn, addr = s.accept()
        print('Conntected with', addr)
        # append client socket on list
        clientsoclist[i] = conn
        args = (i, num_user, conn)
        thread = Thread(target=func, args=args)
        thrs.append(thread)
        thread.start()
    print("timmer start!")
    start_time = time.time()    # store start time
    for thread in thrs:
        thread.join()
    end_time = time.time()  # store end time
    print("TrainingTime: {} sec".format(end_time - start_time))

## Receive client

In [24]:
def receive(userid, num_users, conn): #thread for receive clients
    global weight_count
    


    msg = {
        'epochs': epochs,
        'users': users
    }

    datasize = send_msg(conn, msg)    #send epoch
    total_sendsize_list.append(datasize)
    client_sendsize_list[userid].append(datasize)

    total_batch, datasize = recv_msg(conn)    # get total_batch of train dataset
    total_receivesize_list.append(datasize)
    client_receivesize_list[userid].append(datasize)
    with lock:
        weight_count += 1
    
    train(userid, total_batch, num_users, conn)

## traning

In [25]:
def train(userid, total_batch, num_users, client_conn):
    global client_weights
    global weight_count

    model_num = userid
    
    for e in range(epochs):
        
        for user in range(num_users):
            with lock:
                if weight_count == num_users:
                    interval = model_num - userid
                    for i, conn in enumerate(clientsoclist):
                        datasize = send_msg(conn, client_weights[(i + interval) % num_users])
                        total_sendsize_list.append(datasize)
                        client_sendsize_list[i].append(datasize)
                        train_sendsize_list.append(datasize)
                        weight_count = 0
                        
            for i in range(total_batch):
                optimizer_server_list[model_num].zero_grad()  # initialize all gradients to zero

                msg, datasize = recv_msg(client_conn)  # receive client message from socket
                total_receivesize_list.append(datasize)
                client_receivesize_list[userid].append(datasize)
                train_receivesize_list.append(datasize)

                client_output_cpu = msg['client_output']  # client output tensor
                label = msg['label']  # label

                client_output = client_output_cpu.to(device)
                label = label.clone().detach().long().to(device)

                output = server_models[model_num](client_output)  # forward propagation
                loss = criterion(output, label)  # calculates cross-entropy loss
                loss.backward()  # backward propagation
                msg = client_output_cpu.grad.clone().detach()

                datasize = send_msg(client_conn, msg)
                total_sendsize_list.append(datasize)
                client_sendsize_list[userid].append(datasize)
                train_sendsize_list.append(datasize)

                optimizer_server_list[model_num].step()
                
            
            weights, datasize = recv_msg(client_conn)
            total_receivesize_list.append(datasize)
            client_receivesize_list[userid].append(datasize)
            train_receivesize_list.append(datasize)
            with lock:
                client_weights[model_num] = weights
                weight_count += 1
            model_num = (model_num + 1) % num_users
        print("Epoch {}'s user {} is done".format(e, userid))
        
        
        ecg_client.load_state_dict(client_weights[userid])
        ecg_client.to(device)

        ecg_client_dict = ecg_client.state_dict()
        ecg_server_dict = server_models[userid].state_dict()
        ecg_original_dict = original_models[userid].state_dict()

        ecg_original_dict.update(ecg_client_dict)
        ecg_original_dict.update(ecg_server_dict)

        original_models[userid].load_state_dict(ecg_original_dict)
        original_models[userid].eval()

        # train acc
        with torch.no_grad():
            corr_num = 0
            total_num = 0
            train_loss = 0.0
            for j, trn in enumerate(train_loader):
                trn_x, trn_label = trn
                trn_x = trn_x.to(device)

                trn_label = trn_label.clone().detach().long().to(device)


                trn_output = original_models[userid](trn_x)
                loss = criterion(trn_output, trn_label)

                train_loss += loss.item()
                model_label = trn_output.argmax(dim=1)
                corr = trn_label[trn_label == model_label].size(0)
                corr_num += corr
                total_num += trn_label.size(0)
            train_accuracy = corr_num / total_num * 100
            r_train_loss = train_loss / len(train_loader)
            print("model {}'s train_acc: {:.2f}%, train_loss: {:.4f}".format(userid, train_accuracy, r_train_loss))
            train_acc[userid].append(train_accuracy)

        # test acc
        with torch.no_grad():
            corr_num = 0
            total_num = 0
            val_loss = 0.0
            for j, val in enumerate(test_loader):
                val_x, val_label = val
                val_x = val_x.to(device)
                val_label = val_label.to(device)

                val_output = original_models[userid](val_x)
                val_label = val_label.long()
                loss = criterion(val_output, val_label)
                val_loss += loss.item()
                model_label = val_output.argmax(dim=1)
                corr = val_label[val_label == model_label].size(0)
                corr_num += corr
                total_num += val_label.size(0)
            test_accuracy = corr_num / total_num * 100
            test_loss = val_loss / len(test_loader)
            print("model {}'s test_acc: {:.2f}%, test_loss: {:.4f}".format(userid, test_accuracy, test_loss))
            val_acc[userid].append(test_accuracy)

            
            
    print('{} is complite'.format(userid))
            
                
        

In [26]:
host = socket.gethostbyname(socket.gethostname())
port = 10080
print(host)

192.168.83.1


### Open the server socket

In [27]:
s = socket.socket()
try:
    s.bind((host, port))
    print('Success to connect')
except:
    print('Fail to connect')
    
s.listen(5)

Success to connect


In [28]:
run_thread(receive, users)

Conntected with ('192.168.83.1', 1891)
Conntected with ('192.168.83.1', 1894)
Conntected with ('192.168.83.1', 1897)
timmer start!


C:\Users\rlaal\anaconda3\envs\py36\lib\site-packages\torch\storage.py:34: FutureWarning: pickle support for Storage will be removed in 1.5. Use `torch.save` instead
  warnings.warn("pickle support for Storage will be removed in 1.5. Use `torch.save` instead", FutureWarning)


Epoch 0's user 1 is done
Epoch 0's user 2 is done
Epoch 0's user 0 is done
model 1's train_acc: 74.96%, train_loss: 1.1552
model 2's train_acc: 68.27%, train_loss: 1.2196
model 0's train_acc: 79.61%, train_loss: 1.1105
model 1's test_acc: 76.11%, test_loss: 1.1477
1 is complite
model 2's test_acc: 68.68%, test_loss: 1.2157
2 is complite
model 0's test_acc: 80.51%, test_loss: 1.1018
0 is complite
TrainingTime: 31.657214403152466 sec


## Print Acc

In [ ]:
for i in range(users):
    print('train_acc list(model{})---'.format(i))
    for acc in train_acc[i]:
        print(acc)

    print('val_acc list(model{})---'.format(i))
    for acc in val_acc[i]:
        print(acc)

## Print commmunication overheads 

In [30]:
   
print('\n')
print('---total_sendsize_list---')
total_size = 0
for size in total_sendsize_list:
#     print(size)
    total_size += size
print("total_sendsize size: {} bytes".format(total_size))
print("number of total_send: ", len(total_sendsize_list))
print('\n')

print('---total_receivesize_list---')
total_size = 0
for size in total_receivesize_list:
#     print(size)
    total_size += size
print("total receive sizes: {} bytes".format(total_size) )
print("number of total receive: ", len(total_receivesize_list) )
print('\n')

for i in range(users):
    print('---client_sendsize_list(user{})---'.format(i))
    total_size = 0
    for size in client_sendsize_list[i]:
#         print(size)
        total_size += size
    print("total client_sendsizes(user{}): {} bytes".format(i, total_size))
    print("number of client_send(user{}): ".format(i), len(client_sendsize_list[i]))
    print('\n')

    print('---client_receivesize_list(user{})---'.format(i))
    total_size = 0
    for size in client_receivesize_list[i]:
#         print(size)
        total_size += size
    print("total client_receive sizes(user{}): {} bytes".format(i, total_size))
    print("number of client_send(user{}): ".format(i), len(client_receivesize_list[i]))
    print('\n')

print('---train_sendsize_list---')
total_size = 0
for size in train_sendsize_list:
#     print(size)
    total_size += size
print("total train_sendsizes: {} bytes".format(total_size))
print("number of train_send: ", len(train_sendsize_list) )
print('\n')

print('---train_receivesize_list---')
total_size = 0
for size in train_receivesize_list:
#     print(size)
    total_size += size
print("total train_receivesizes: {} bytes".format(total_size))
print("number of train_receive: ", len(train_receivesize_list) )
print('\n')


train_acc list(model0)---
79.60739901849755
val_acc list(model0)---
80.50585126462816
train_acc list(model1)---
74.95658739146847
val_acc list(model1)---
76.1117402793507
train_acc list(model2)---
68.27482068705172
val_acc list(model2)---
68.68252170630427


---total_sendsize_list---
total_sendsize size: 165815787 bytes
number of total_send:  1254


---total_receivesize_list---
total receive sizes: 166380693 bytes
number of total receive:  1254


---client_sendsize_list(user0)---
total client_sendsizes(user0): 55271929 bytes
number of client_send(user0):  418


---client_receivesize_list(user0)---
total client_receive sizes(user0): 55460231 bytes
number of client_send(user0):  418


---client_sendsize_list(user1)---
total client_sendsizes(user1): 55271929 bytes
number of client_send(user1):  418


---client_receivesize_list(user1)---
total client_receive sizes(user1): 55460231 bytes
number of client_send(user1):  418


---client_sendsize_list(user2)---
total client_sendsizes(user2): 55

# Validation after trainning

### acc of each acc 

In [32]:
classes = ['N', 'L', 'R', 'A', 'V']


for n, model in enumerate(original_models):
    class_correct = list(0. for i in range(5))
    class_total = list(0. for i in range(5))
    with torch.no_grad():
        for data in test_loader:
            x, labels = data
            x = x.to(device)
            labels = labels.to(device)

            outputs = model(x)
            labels = labels.long()
            _, predicted = torch.max(outputs, 1)
            c = (predicted == labels).squeeze()
            for i in range(len(labels)):
                label = labels[i]
                class_correct[label] += c[i].item()
                class_total[label] += 1


    for i in range(5):
        print("model %d's Accuracy of %5s : %2d %%" % (n, 
            classes[i], 100 * class_correct[i] / class_total[i]))
    print('\n')


model 0's Accuracy of     N : 95 %
model 0's Accuracy of     L : 97 %
model 0's Accuracy of     R : 72 %
model 0's Accuracy of     A :  0 %
model 0's Accuracy of     V : 90 %


model 1's Accuracy of     N : 93 %
model 1's Accuracy of     L : 77 %
model 1's Accuracy of     R : 77 %
model 1's Accuracy of     A :  0 %
model 1's Accuracy of     V : 87 %


model 2's Accuracy of     N : 44 %
model 2's Accuracy of     L : 98 %
model 2's Accuracy of     R : 77 %
model 2's Accuracy of     A :  0 %
model 2's Accuracy of     V : 82 %




In [33]:
# Let's quickly save our trained model:
for i, model in enumerate(original_models):
    PATH = './ecg_es_model{}.pth'.format(i)
    torch.save(model.state_dict(), PATH)

In [34]:
end_time = time.time()  # store end time
print("WorkingTime: {} sec".format(end_time - start_time))

WorkingTime: 57.16648530960083 sec
